Okay so the last thing was fine but was getting pretty complex and unweildy having to deal with all these edge cases.

I think now what I want to do is just create an agent whose job it is to fill out some tables, and just let math do the rest

Have the AI fill out the following information

Expenses
[
- Name
- Total
]

For each of these expenses

Expense (paid by who and how much)
[
- Name
- Amount
]

Expense (used by who and how much value)
[
- Name
- Amount 
]

Then we can pass that information into a resolve_debtor_collector function which
- sums how much each person paid
- sums how much each person used
- Ordered debtors and collectors
- Iteratively creates a list of who should pay whom until debts are paid off




In [177]:
inp_simple = "Joe spent $5 and Hannah spent $8 on a toilet plunger"
inp_complex = """Sarah bought groceries for $60 and a bottle of wine for $30.
Jake paid for gas, which costs $50, and bought snacks for $25.
Emily booked $150 hotel room for her and Sarah and paid for dinner, which was $90.
Jake booked his own room for $80, and also bought a hotdog for Emily for $15.
Jake and Sarah also split a $20 popcorn which they shared with Emily"""

### Create some basic tools the LLM should be able to use

In [178]:
from langchain_core.tools import tool
from typing import List
from langchain_core.pydantic_v1 import BaseModel, Field

@tool
def add(first_float: float, second_float: float) -> float:
    """Adds two floats"""
    return first_float + second_float

@tool
def subtract(first_float: float, second_float: float) -> float:
    """Subtracts first_float from second_float"""
    return first_float - second_float

@tool
def divide(dividend: float, divisor: int) -> float:
    """Divides dividend by divisor."""
    return dividend / divisor

@tool
def multiply(float_factor: float, int_factor: int) -> float:
    """Multiplies float_factor by int_factor."""
    return float_factor * int_factor

math_tools = [add, subtract, divide, multiply]

In [179]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.prompts import MessagesPlaceholder

llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0125")

In [205]:
list_items_template = """You will be provided a passage of someone describing purchase(s) made together.

Thoroughly list each unique purchase made.

Respond with a comma-separated list of '<item name> (<buyers>)'.

Passage:
{input}
"""

list_items_prompt = ChatPromptTemplate.from_template(list_items_template)

list_items_chain = list_items_prompt | llm

In [206]:
list_items_chain.invoke(inp_simple)

AIMessage(content='Toilet plunger (Joe, Hannah)', response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 65, 'total_tokens': 74}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-437bdd54-a383-4f24-ac3c-c93a9740b117-0')

In [207]:
list_items_chain.invoke(inp_complex)

AIMessage(content='Groceries (Sarah), Bottle of wine (Sarah), Gas (Jake), Snacks (Jake), Hotel room (Emily), Dinner (Emily), Room (Jake), Hotdog (Jake), Popcorn (Jake, Sarah)', response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 138, 'total_tokens': 183}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-1d4b0546-4741-4af5-8d9d-3a7a16a40c28-0')

In [240]:
system_prompt = """You are a helpful accounting assistant that is skilled at using
math tools to answer questions.

IMPORTANT: use the tools provided to do any calculations.
"""

human_prompt = """
Given the list of items below and who bought them, reference the passage below to
calculate the individual contributions to the purchase and the total cost.

Please use the tools provided to do any calculations.

List:
{items_list}


Passage:
{input}

Respond with a newline separated list, in the following format:

"<people> bought <purchase> for a total of <cost>, with <person 1> paying <person 1 contribution>..."
"""


prompt = ChatPromptTemplate.from_messages(
[
    ("system", system_prompt),
    ("human", human_prompt),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
]
)

In [241]:
from langchain.agents import create_openai_functions_agent

agent = create_openai_functions_agent(llm, math_tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=math_tools, verbose=True)

In [242]:
list(agent_executor.stream({"input": inp_simple, "items_list": "Toilet plunger (Joe, Hannah)"}))



> Entering new AgentExecutor chain...

Invoking: `add` with `{'first_float': 5, 'second_float': 8}`


13.0Joe and Hannah bought a toilet plunger for a total of $13, with Joe paying $5 and Hannah paying $8.

> Finished chain.


[{'actions': [AgentActionMessageLog(tool='add', tool_input={'first_float': 5, 'second_float': 8}, log="\nInvoking: `add` with `{'first_float': 5, 'second_float': 8}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'function_call': {'arguments': '{"first_float":5,"second_float":8}', 'name': 'add'}}, response_metadata={'finish_reason': 'function_call'}, id='run-069764b8-b249-465b-a7c7-fd258ffb9d14')])],
  'messages': [AIMessageChunk(content='', additional_kwargs={'function_call': {'arguments': '{"first_float":5,"second_float":8}', 'name': 'add'}}, response_metadata={'finish_reason': 'function_call'}, id='run-069764b8-b249-465b-a7c7-fd258ffb9d14')]},
 {'steps': [AgentStep(action=AgentActionMessageLog(tool='add', tool_input={'first_float': 5, 'second_float': 8}, log="\nInvoking: `add` with `{'first_float': 5, 'second_float': 8}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'function_call': {'arguments': '{"first_float":5,"second_float":8}', 'name': 

In [243]:
list(agent_executor.stream({"input": inp_complex, "items_list": "Groceries (Sarah), Bottle of wine (Sarah), Gas (Jake), Snacks (Jake), Hotel room (Emily), Dinner (Emily), Room (Jake), Hotdog (Jake), Popcorn (Jake, Sarah)"}))



> Entering new AgentExecutor chain...
Sarah bought Groceries for a total of $60, with Sarah paying $60.
Sarah bought a Bottle of wine for a total of $30, with Sarah paying $30.
Jake bought Gas for a total of $50, with Jake paying $50.
Jake bought Snacks for a total of $25, with Jake paying $25.
Emily bought a Hotel room for a total of $150, with Emily paying $75.
Emily bought Dinner for a total of $90, with Emily paying $45.
Jake bought a Room for a total of $80, with Jake paying $40.
Jake bought a Hotdog for a total of $15, with Jake paying $15.
Jake and Sarah bought Popcorn for a total of $20, with Jake paying $10 and Sarah paying $10.

> Finished chain.


[{'output': 'Sarah bought Groceries for a total of $60, with Sarah paying $60.\nSarah bought a Bottle of wine for a total of $30, with Sarah paying $30.\nJake bought Gas for a total of $50, with Jake paying $50.\nJake bought Snacks for a total of $25, with Jake paying $25.\nEmily bought a Hotel room for a total of $150, with Emily paying $75.\nEmily bought Dinner for a total of $90, with Emily paying $45.\nJake bought a Room for a total of $80, with Jake paying $40.\nJake bought a Hotdog for a total of $15, with Jake paying $15.\nJake and Sarah bought Popcorn for a total of $20, with Jake paying $10 and Sarah paying $10.',
  'messages': [AIMessage(content='Sarah bought Groceries for a total of $60, with Sarah paying $60.\nSarah bought a Bottle of wine for a total of $30, with Sarah paying $30.\nJake bought Gas for a total of $50, with Jake paying $50.\nJake bought Snacks for a total of $25, with Jake paying $25.\nEmily bought a Hotel room for a total of $150, with Emily paying $75.\nEm

In [245]:
system_prompt = """You are a helpful accounting assistant that is skilled at checking
the work of other accountants."""

human_prompt = """
You will be passed a list of purchases, who contributed towards paying for the purchase, and how much they contributed.

Using the information in the passage, use the math tools provided to perform calculations verifying the
total cost of the item and individual contributions towards the total.

List:
{item_contributions_list}

Passage:
{input}


You should edit any listed information deemed incorrect by the calculations in the same format.
"""


prompt = ChatPromptTemplate.from_messages(
[
    ("system", system_prompt),
    ("human", human_prompt),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

In [246]:
agent = create_openai_functions_agent(llm, math_tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=math_tools, verbose=True)

In [248]:
list(agent_executor.stream({"input": inp_simple, "item_contributions_list": "Joe and Hannah bought a toilet plunger for a total of $13, with Joe paying $5 and Hannah paying $8."}))



> Entering new AgentExecutor chain...


KeyboardInterrupt: 